<a href="https://colab.research.google.com/github/gyeomsoo/Homework-2025/blob/main/192_Display_on_the_map_of_the_earthquake_area_193_Displaying_on_a_Weather_Information_Map.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 192 지진 발생 지역 지도에 표시하기

In [ ]:
from urllib.request import urlopen
url = 'https://earthquake.usgs.gov/fdsnws/event/1/query?format=csv&minmag-nitude=3.0'
with urlopen(url) as f:
  print(f.read().decode())

In [ ]:
from mpl_tookits.basemap import Basemap
import matplotlib.pyplot as plt
import matplotlib as mpl
import numpy as np
from urllib.request import urlopen
from time import localtime, strftime
from math import sqrt
def readEarthquakeData(minmag=3):
  lats = [ ]; lons = [ ]; mags = [ ]
  query = f'format = csv&minmagnitude={minmag}'
  url = f'https://earthquake.wsgs.gov/fdsnws/event/1/query?{query}'
  with urlopen(url) as f:
    earthquake_data = f.read().decode()
    data = derthquake_data.split('\n')
    del data[0]
    for d in data:
      tmp = d.split(',')
      if len(tmp) > 4:
        lats.append(float(tmp[1]))
        lons.append(float(tmp[2]))
        mags.append(float(tmp[4]))

  return lats, lons, mags

def getMarkerColor(mag):
  if mag < 4.0:
    return 'bo'
  elif mag < 5.0:
    return 'go'
  elif mag < 6.0:
    return 'yo'
  elif mag < 7.0:
    return 'mo'
  else:
    return 'ro'

def drawMap(minmag):
  mpl.style.use('seaborn-v0_8-talk')
  krfont = {'family':'NanumGothic', 'weight':'bold', 'size':10}
  mpl.rc('font', **krfont)
  mpl.rcParams['axes.unicode_minus'] = False

  m = Basemap(projection='robin', resolution='l', lat_0=0, lon_0=128)
  m.drawcoastlines()
  m.drawcountries()
  m.fillcontinents(color='gray')
  m.drawmeridians(np.arange(0, 360, 30))
  m.drawparallels(np.arange(-90, 90, 30))

  lats, lons, mags = readEarthquakeData(minmag)
  for lat, lon, mag in zip(lats, lons, mags):
    x, y = m(lon, lat)
    color = getMarkerColor(mag)
    m.plot(x, y, color, markersize=sqrt(mag)*mag, alpha=0.7)

  time_stamp = strftime('%Y-%m-%d', localtime())
  title = f'{time_stamp} 기준 1개월간 전 세계 규모 {minmag} 이상 지진 발생 현황'
  plt.title(title)
  plt.show()

if __name__=='__main__':
  minmag = input('조회하고 싶은 최소 지진 규모를 입력하세요: ')
  try:
    minmag = float(minmag)
    if minmag < 0:
      print(f'입력하신 지진 규모가 0보다 작습니다.')
    else:
      drawMap(minmag)
  except Exception as e:
    print(f'오류 발생: {e}')

# 193 기상 정보 지도에 표시하기

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
from mpl_toolkits.basemap import Basemap
import numpy as np
from datetime import datetime, timedelta
import netCDF4 as nc

def getAirData():
  ncfile = 'air.sig995.2024.nc'
  f = nc.Dataset(ncfile)
  air = f['air']
  endidx = len(air) - 1

  while True:
    idx = int(input(f'0~{endidx} 사이 숫자를 입력하세요: '))
    if idx < 0 or idx > endidx:
      print(f'{idx} : 0~{endidx} 사이의 숫자가 아닙니다.')
      continue
    else:
      break

  airtemp = air[:] - 273.15
  airtemp = airtemp[idx,::]
  lats = f['lat'][:]
  lons = f['lon'][:]
  elapsed_hours = f['time'][:][idx]
  datedata = datetime(1800, 1, 1) + timedelta(hours = elapsed_hours+9)

  return airtemp, lats, lons, datedata

def drawMap():
  mpl.style.use('seaborn-v0_8-talk')
  krfont = {'family':'NanumGothic', 'weight':'bold', 'size':10}
  mpl.rc('font', **krfont)
  mpl.rcParams['axes.unicode_minus'] = False

  airtemp, lats, lons, datedata = getAirData()

  m = Basemap(projection='ortho', lat_0=35, lon_0=128, resolution='1')
  lon2, lat2 = np.meshgrid(lons, lats)
  x, y = m(lon2, lat2)

  m.drawcoastlines()
  m.drawcountries()
  m.drawmapboundary(fill_color='white')
  m.drawmeridians(np.arange(0, 360, 30))
  m.drawparallels(np.arange(-90, 90, 30))

  cs = m.contourf(x, y, airtemp, 20, cmap=mpl.colormaps.get_cmap('jet'))
  plt.colotbar()

  title = f'{datedata} 평균 기온 분포도'
  plt.title(title)
  plt.show()

if __name__=='__main__':
  drawMap()